# BKM:
*****
>1. Config slaves as "cluster palcement group", to get best and stable throughput among nodes.
2. Open port accesses internally, don't expose port access to public
3. Configure security.authorization and ip control through hadoop
4. Network throughput depends on instance type. If master node is small instance, avoid to copy large file from master to slaves or HDFS, using one of slavers
5. If you want to cache the files to memory, set replication to 1. Otherwise you can't make sure Yarn schedule the same task to the same node always.
6. If you copy large file from one slave to whole HDFS, the distribution is biased. The slave holds most of the data in its HDFS. Solution is to copy the file from HDFS to HDFS again
7. vcpu isn't map to physical CPU with the same index, so you can't make sure two vcpu doesn't share the same physical core. So pin executors to cores through the native OS policy leads to poor performance. We need to make clear how vcpu share the same core firstly
8. 
*****

# Command from shell

# Initialize <font color=red> Run this section after note book restart! </font>

In [1]:
#datadir=['/data1','/data2','/data3','/data4']
datadir=['/home/sparkuser']

In [4]:
clients=''' '''.split()
clients

[]

In [5]:
import socket
masterip=socket.gethostbyname(socket.gethostname())
hostname=socket.gethostname()   


In [6]:
hostname

'devrestricted-binweiyang'

In [7]:
hclients=clients.copy()
hclients.append(hostname)

In [8]:
hclients

['devrestricted-binweiyang']

In [9]:
user="sparkuser"

In [10]:
%cd ~

/home/sparkuser


In [11]:
import os
cpu_num=os.cpu_count()

meminfo = dict((i.split()[0].rstrip(':'),int(i.split()[1])) for i in open('/proc/meminfo').readlines())
mem_mib = int(int(meminfo['MemTotal'])/1024)-1024
print(cpu_num,mem_mib)

8 62496


In [12]:
sparklocals=",".join([f'{l}/{user}/hdfs/yarn/local' for l in datadir])

In [13]:
datadir

['/home/sparkuser']

In [14]:
import platform

is_arm = platform.machine() == 'aarch64'
print("is_arm: ",is_arm)

is_arm:  False


In [15]:
totalcpu=!grep 'processor' /proc/cpuinfo | wc -l
totalcpu=int(totalcpu[0])
print("total cpu number: ", totalcpu)

total cpu number:  8


In [16]:
# for gluten, suggest partition as 2x cpu#
scaleFactor = 1000
numPartitions = 2*totalcpu if len(clients)==0 else len(clients)*2*totalcpu
print("partitions number", numPartitions)

partitions number 16


In [17]:
rootDir = "/tpcds_sf1000_zstd" # root directory of location to create data in.
dataformat = "parquet" # data format of data source

In [18]:
dataSourceCodec = "zstd"

In [19]:
num_executors = 8
executor_cores=totalcpu/num_executors

totalmemory = !cat /proc/meminfo | grep MemTotal | awk '{print $2}'
totalmemory = int(totalmemory[0])
print("total memory is ", totalmemory, "KB")
executor_memory = (totalmemory - 3*1024*1024)/num_executors

total memory is  65045400 KB


# partition worker node <font color=red> SKIP for single node </font>

## partition and mount disks

In [ ]:
!lsblk

In [ ]:
blks=['/dev/nvme2n1']

In [ ]:
if len(blks) != len(datadir) - 1:
    print("blks length and datadir length doesn't match!")

In [ ]:
for l in hclients:
    for b in blks:
        !ssh root@{l} "echo -e \"g\\nw\\n\" | fdisk {b}"
        !ssh root@{l} "echo -e \"n\\n\\n\\n\\nw\\n\" | fdisk {b}"
        !ssh root@{l} "mkfs.ext4 {b}p1"

In [ ]:
!lsblk

In [ ]:
for l in hclients:
    for b, d in zip(blks, datadir[1:]):
        !ssh root@{l} "mkdir -p {d}"
        !ssh root@{l} "mount {b}p1 {d}"

In [ ]:
!mount -l

## install python

In [ ]:
!pip install findspark

!pip install pandas numpy

!pip install matplotlib

!pip install pandasql

# create user<font color=red> SKIP for single node </font>

In [ ]:
for l in clients:
    !ssh root@$l adduser {user}

In [ ]:
for i in clients:
    !ssh root@$i cp -r .ssh /home/{user}/
    !ssh root@$i chown -R {user}:{user} /home/{user}/.ssh

# Download Spark Hadoop

In [17]:
%cd ~

/home/sparkuser


In [21]:
!wget https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

--2023-09-23 05:12:42--  https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299360284 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.2-bin-hadoop3.tgz’

spark-3.3.2-bin-had 100%[===================>] 285.49M  27.3MB/s    in 11s     

2023-09-23 05:12:54 (25.7 MB/s) - ‘spark-3.3.2-bin-hadoop3.tgz’ saved [299360284/299360284]



In [19]:
#download both version

!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.2.4/hadoop-3.2.4.tar.gz > /dev/null 2>&1
if is_arm:
    !wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.5/hadoop-3.3.5-aarch64.tar.gz > /dev/null 2>&1
    

# Create directories clients

In [48]:
cmd=";".join([f"chown -R {user}:{user} " + l for l in datadir])
!{cmd}

In [49]:
cmd=";".join([f"rm -rf {l}/tmp; mkdir -p {l}/tmp" for l in datadir])
!{cmd}

In [50]:
cmd

'rm -rf /home/sparkuser/tmp; mkdir -p /home/sparkuser/tmp'

In [51]:
cmd=";".join([f"mkdir -p {l}/{user}/hdfs/name; mkdir -p {l}/{user}/hdfs/data; mkdir -p {l}/{user}/yarn; mkdir -p {l}/{user}/yarn/local" for l in datadir])
!{cmd}

In [52]:
!mkdir -p {datadir[0]}/{user}/hdfs/namesecondary

In [55]:
%pwd

'/home/sparkuser'

In [53]:
!mv -f hadoop hadoop.bak; mv -f spark spark.bak
!tar zxvf hadoop-3.2.4.tar.gz > /dev/null 2>&1
!tar -zxvf spark-3.3.2-bin-hadoop3.tgz > /dev/null 2>&1
!sudo apt install openjdk-8-jdk > /dev/null 2>&1
!ln -s hadoop-3.2.4 hadoop; ln -s spark-3.3.2-bin-hadoop3 spark
if is_arm:
    !tar zxvf hadoop-3.3.5-aarch64.tar.gz > /dev/null 2>&1
    !cd hadoop && mv lib lib.bak && cp -rf ~/hadoop-3.3.5/lib ~/hadoop

mv: cannot stat 'hadoop': No such file or directory
mv: cannot stat 'spark': No such file or directory


# Config bashrc

In [57]:

cfg=f'''export HADOOP_HOME=~/hadoop
export PATH=$PATH:$HADOOP_HOME/bin:$HADOOP_HOME/sbin

export HADOOP_CONF_DIR=$HADOOP_HOME/etc/hadoop
export YARN_CONF_DIR=$HADOOP_HOME/etc/hadoop

export PATH=$JAVA_HOME/bin:$PATH
export SPARK_HOME=~/spark
export PYTHONPATH=$SPARK_HOME/python:$SPARK_HOME/python/lib/py4j-0.10.9.5-src.zip:$PYTHONPATH
export PATH=$SPARK_HOME/bin:$PATH

'''

In [58]:
if is_arm:
    cfg += 'export CPU_TARGET="aarch64"\nexport JAVA_HOME=/usr/lib/jvm/java-8-openjdk-arm64\nexport PATH=$JAVA_HOME/bin:$PATH\n'
else:
    cfg += 'export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64\nexport PATH=$JAVA_HOME/bin:$PATH\n'

In [59]:
with open("tmpcfg",'w') as f:
    f.writelines(cfg)

In [61]:
!cat ~/tmpcfg >> ~/.bashrc

In [62]:
!tail ~/.bashrc

export HADOOP_CONF_DIR=$HADOOP_HOME/etc/hadoop
export YARN_CONF_DIR=$HADOOP_HOME/etc/hadoop

export PATH=$JAVA_HOME/bin:$PATH
export SPARK_HOME=~/spark
export PYTHONPATH=$SPARK_HOME/python:$SPARK_HOME/python/lib/py4j-0.10.9.5-src.zip:$PYTHONPATH
export PATH=$SPARK_HOME/bin:$PATH

export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
export PATH=$JAVA_HOME/bin:$PATH


# config Hadoop

In [63]:
!sudo apt install -y libiberty-dev libxml2-dev libkrb5-dev libgsasl7-dev libuuid1 uuid-dev > /dev/null 2>&1

## setup  short-circuit 

In [65]:
!sudo mkdir -p /var/lib/hadoop-hdfs/
!sudo chown {user}:{user} /var/lib/hadoop-hdfs/

## enable security.authorization

In [66]:
coresite='''<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
      <name>fs.default.name</name>
      <value>hdfs://{:s}:8020</value>
      <final>true</final>
  </property>
  <property>
      <name>hadoop.security.authentication</name>
      <value>simple</value>
  </property>
  <property>
      <name>hadoop.security.authorization</name>
      <value>true</value>
  </property>
</configuration>
'''.format(hostname)

with open('hadoop/etc/hadoop/core-site.xml','w') as f:
    f.writelines(coresite)
    
for l in clients:
    !scp hadoop/etc/hadoop/core-site.xml $l:~/hadoop/etc/hadoop/core-site.xml >/dev/null 2>&1

## set IP check, note the command <font color=red>", "</font>.join

In [67]:
hostname

'devrestricted-binweiyang'

In [68]:
hadooppolicy='''<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--

 Licensed to the Apache Software Foundation (ASF) under one
 or more contributor license agreements.  See the NOTICE file
 distributed with this work for additional information
 regarding copyright ownership.  The ASF licenses this file
 to you under the Apache License, Version 2.0 (the
 "License"); you may not use this file except in compliance
 with the License.  You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.

-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
    <name>security.service.authorization.default.hosts</name>
    <value>{:s}</value>
  </property>
  <property>
    <name>security.service.authorization.default.acl</name>
    <value>{:s} {:s}</value>
  </property>
  
  
    <property>
    <name>security.client.protocol.acl</name>
    <value>*</value>
    <description>ACL for ClientProtocol, which is used by user code
    via the DistributedFileSystem.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.client.datanode.protocol.acl</name>
    <value>*</value>
    <description>ACL for ClientDatanodeProtocol, the client-to-datanode protocol
    for block recovery.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.datanode.protocol.acl</name>
    <value>*</value>
    <description>ACL for DatanodeProtocol, which is used by datanodes to
    communicate with the namenode.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.inter.datanode.protocol.acl</name>
    <value>*</value>
    <description>ACL for InterDatanodeProtocol, the inter-datanode protocol
    for updating generation timestamp.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.namenode.protocol.acl</name>
    <value>*</value>
    <description>ACL for NamenodeProtocol, the protocol used by the secondary
    namenode to communicate with the namenode.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

 <property>
    <name>security.admin.operations.protocol.acl</name>
    <value>*</value>
    <description>ACL for AdminOperationsProtocol. Used for admin commands.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.refresh.user.mappings.protocol.acl</name>
    <value>*</value>
    <description>ACL for RefreshUserMappingsProtocol. Used to refresh
    users mappings. The ACL is a comma-separated list of user and
    group names. The user and group list is separated by a blank. For
    e.g. "alice,bob users,wheel".  A special value of "*" means all
    users are allowed.</description>
  </property>

  <property>
    <name>security.refresh.policy.protocol.acl</name>
    <value>*</value>
    <description>ACL for RefreshAuthorizationPolicyProtocol, used by the
    dfsadmin and mradmin commands to refresh the security policy in-effect.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.ha.service.protocol.acl</name>
    <value>*</value>
    <description>ACL for HAService protocol used by HAAdmin to manage the
      active and stand-by states of namenode.</description>
  </property>

  <property>
    <name>security.zkfc.protocol.acl</name>
    <value>*</value>
    <description>ACL for access to the ZK Failover Controller
    </description>
  </property>

  <property>
    <name>security.qjournal.service.protocol.acl</name>
    <value>*</value>
    <description>ACL for QJournalProtocol, used by the NN to communicate with
    JNs when using the QuorumJournalManager for edit logs.</description>
  </property>

  <property>
    <name>security.mrhs.client.protocol.acl</name>
    <value>*</value>
    <description>ACL for HSClientProtocol, used by job clients to
    communciate with the MR History Server job status etc.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <!-- YARN Protocols -->

  <property>
    <name>security.resourcetracker.protocol.acl</name>
    <value>*</value>
    <description>ACL for ResourceTrackerProtocol, used by the
    ResourceManager and NodeManager to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.resourcemanager-administration.protocol.acl</name>
    <value>*</value>
    <description>ACL for ResourceManagerAdministrationProtocol, for admin commands.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.applicationclient.protocol.acl</name>
    <value>*</value>
    <description>ACL for ApplicationClientProtocol, used by the ResourceManager
    and applications submission clients to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.applicationmaster.protocol.acl</name>
    <value>*</value>
    <description>ACL for ApplicationMasterProtocol, used by the ResourceManager
    and ApplicationMasters to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.containermanagement.protocol.acl</name>
    <value>*</value>
    <description>ACL for ContainerManagementProtocol protocol, used by the NodeManager
    and ApplicationMasters to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.resourcelocalizer.protocol.acl</name>
    <value>*</value>
    <description>ACL for ResourceLocalizer protocol, used by the NodeManager
    and ResourceLocalizer to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.job.task.protocol.acl</name>
    <value>*</value>
    <description>ACL for TaskUmbilicalProtocol, used by the map and reduce
    tasks to communicate with the parent tasktracker.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.job.client.protocol.acl</name>
    <value>*</value>
    <description>ACL for MRClientProtocol, used by job clients to
    communciate with the MR ApplicationMaster to query job status etc.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  <property>
    <name>security.applicationhistory.protocol.acl</name>
    <value>*</value>
    <description>ACL for ApplicationHistoryProtocol, used by the timeline
    server and the generic history service client to communicate with each other.
    The ACL is a comma-separated list of user and group names. The user and
    group list is separated by a blank. For e.g. "alice,bob users,wheel".
    A special value of "*" means all users are allowed.</description>
  </property>

  
  
  
  
</configuration>
'''.format((", ").join(hclients),user,user)

with open('hadoop/etc/hadoop/hadoop-policy.xml','w') as f:
    f.writelines(hadooppolicy)
    
for l in clients:
    !scp hadoop/etc/hadoop/hadoop-policy.xml $l:~/hadoop/etc/hadoop/hadoop-policy.xml >/dev/null 2>&1


## hdfs config, set replication to 1 to cache all the data

In [69]:
hdfs_name=",".join([f'{l}/{user}/hdfs/name' for l in datadir])
hdfs_data=",".join([f'{l}/{user}/hdfs/data' for l in datadir])

hdfs_site=f'''<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
    <name>dfs.namenode.secondary.http-address</name>
    <value>{hostname}:50090</value>
  </property>
  <property>
    <name>dfs.namenode.name.dir</name>
    <value>{hdfs_name}</value>
    <final>true</final>
  </property>

  <property>
    <name>dfs.datanode.data.dir</name>
        <value>{hdfs_data}</value>
    <final>true</final>
  </property>

  <property>
    <name>dfs.namenode.checkpoint.dir</name>
    <value>{datadir[0]}/{user}/hdfs/namesecondary</value>
    <final>true</final>
  </property>
  <property>
    <name>dfs.name.handler.count</name>
    <value>100</value>
  </property>
  <property>
    <name>dfs.blocksize</name>
    <value>128m</value>
</property>
  <property>
    <name>dfs.replication</name>
    <value>1</value>
</property>

<property>
   <name>dfs.client.read.shortcircuit</name>
   <value>true</value>
</property>

<property>
   <name>dfs.domain.socket.path</name>
   <value>/var/lib/hadoop-hdfs/dn_socket</value>
</property>

</configuration>
'''


with open('hadoop/etc/hadoop/hdfs-site.xml','w') as f:
    f.writelines(hdfs_site)
    
for l in clients:
    !scp hadoop/etc/hadoop/hdfs-site.xml $l:~/hadoop/etc/hadoop/hdfs-site.xml >/dev/null 2>&1


## mapreduce config

In [70]:
mapreduce='''<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
    <property>
        <name>mapreduce.framework.name</name>
        <value>yarn</value>
    </property>

     <property>
         <name>mapreduce.job.maps</name>
         <value>288</value>
     </property>
     <property>
         <name>mapreduce.job.reduces</name>
         <value>64</value>
     </property>

     <property>
         <name>mapreduce.map.java.opts</name>
         <value>-Xmx5120M -DpreferIPv4Stack=true</value>
     </property>
      <property>
         <name>mapreduce.map.memory.mb</name>
         <value>6144</value>
         </property>

     <property>
         <name>mapreduce.reduce.java.opts</name>
         <value>-Xmx5120M -DpreferIPv4Stack=true</value>
     </property>
     <property>
         <name>mapreduce.reduce.memory.mb</name>
         <value>6144</value>
     </property>
     <property>
         <name>yarn.app.mapreduce.am.staging-dir</name>
         <value>/user</value>
     </property>
     <property>
         <name>mapreduce.task.io.sort.mb</name>
         <value>2000</value>
     </property>
     <property>
         <name>mapreduce.task.timeout</name>
         <value>3600000</value>
     </property>
<!-- MapReduce Job History Server security configs -->
<property>
  <name>mapreduce.jobhistory.address</name>
  <value>{:s}:10020</value>
</property>

</configuration>
'''.format(hostname)


with open('hadoop/etc/hadoop/mapred-site.xml','w') as f:
    f.writelines(mapreduce)
    
for l in clients:
    !scp hadoop/etc/hadoop/mapred-site.xml $l:~/hadoop/etc/hadoop/mapred-site.xml >/dev/null 2>&1


## yarn config

In [71]:
%cd ~

/home/sparkuser


In [72]:
yarn_site=f'''<?xml version="1.0"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->
<configuration>
  <property>
      <name>yarn.resourcemanager.hostname</name>
      <value>{hostname}</value>
  </property>
  <property>
      <name>yarn.resourcemanager.address</name>
      <value>{hostname}:8032</value>
  </property>
  <property>
      <name>yarn.nodemanager.resource.memory-mb</name>
      <value>{mem_mib}</value>
  </property>
  <property>
      <name>yarn.nodemanager.resource.cpu-vcores</name>
      <value>{cpu_num}</value>
  </property>
  <property>
      <name>yarn.nodemanager.pmem-check-enabled</name>
      <value>false</value>
  </property>

  <property>
      <name>yarn.nodemanager.vmem-check-enabled</name>
      <value>false</value>
  </property>
  <property>
      <name>yarn.nodemanager.vmem-pmem-ratio</name>
      <value>4.1</value>
  </property>
  <property>
      <name>yarn.nodemanager.aux-services</name>
      <value>mapreduce_shuffle,spark_shuffle</value>
  </property>

  <property>
      <name>yarn.scheduler.minimum-allocation-mb</name>
      <value>1024</value>
  </property>
  <property>
      <name>yarn.scheduler.maximum-allocation-mb</name>
      <value>{mem_mib}</value>
  </property>
  <property>
      <name>yarn.scheduler.minimum-allocation-vcores</name>
      <value>1</value>
  </property>
  <property>
      <name>yarn.scheduler.maximum-allocation-vcores</name>
      <value>{cpu_num}</value>
  </property>

  <property>
      <name>yarn.log-aggregation-enable</name>
      <value>false</value>
  </property>
  <property>
      <name>yarn.nodemanager.log.retain-seconds</name>
      <value>36000</value>
  </property>
  <property>
      <name>yarn.nodemanager.delete.debug-delay-sec</name>
      <value>3600</value>
  </property>
  <property>
      <name>yarn.log.server.url</name>
      <value>http://{hostname}:19888/jobhistory/logs/</value>
  </property>

  <property>
      <name>yarn.nodemanager.log-dirs</name>
      <value>/home/{user}/hadoop/logs/userlogs</value>
  </property>
  <property>
      <name>yarn.nodemanager.local-dirs</name>
      <value>{sparklocals}
      </value>
  </property>
  <property>
      <name>yarn.nodemanager.aux-services.spark_shuffle.class</name>
      <value>org.apache.spark.network.yarn.YarnShuffleService</value>
  </property>
</configuration>
'''


with open('hadoop/etc/hadoop/yarn-site.xml','w') as f:
    f.writelines(yarn_site)
    
for l in clients:
    !scp hadoop/etc/hadoop/yarn-site.xml $l:~/hadoop/etc/hadoop/yarn-site.xml >/dev/null 2>&1


## hadoop-env

In [73]:
#config java home
if is_arm:
    !echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-arm64" >> ~/hadoop/etc/hadoop/hadoop-env.sh
else:
    !echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64" >> ~/hadoop/etc/hadoop/hadoop-env.sh

for l in clients:
    !scp hadoop/etc/hadoop/hadoop-env.sh $l:~/hadoop/etc/hadoop/ >/dev/null 2>&1


## slave config

In [74]:
with open('hadoop/etc/hadoop/slaves','w') as f:
    f.writelines("\n".join(hclients))

for l in clients:
    !scp hadoop/etc/hadoop/slaves $l:~/hadoop/etc/hadoop/ >/dev/null 2>&1


In [76]:
!cp spark/yarn/spark-3.3.2-yarn-shuffle.jar ~/hadoop/share/hadoop/common/lib/

# config spark

In [77]:
eventlog_dir=f'hdfs://{hostname}:8020/tmp/sparkEventLog'

In [78]:
sparkconf=f'''
spark.eventLog.enabled    true
spark.eventLog.dir        {eventlog_dir}
spark.history.fs.logDirectory    {eventlog_dir}
'''

%cd ~
with open('spark/conf/spark-defaults.conf','w+') as f:
    f.writelines(sparkconf)
    
for l in clients:
    !scp ~/spark/conf/spark-defaults.conf $l:~/spark/conf/spark-defaults.conf >/dev/null 2>&1

/home/sparkuser


In [81]:
sparklocals

'/home/sparkuser/sparkuser/hdfs/yarn/local'

In [79]:
sparkenv = f'export SPARK_LOCAL_DIRS={sparklocals}\n'
with open('/home/sparkuser/.bashrc', 'a+') as f:
    f.writelines(sparkenv)
for l in clients:
    !scp ~/.bashrc $l:~/.bashrc >/dev/null 2>&1

In [80]:
!tail ~/.bashrc

export YARN_CONF_DIR=$HADOOP_HOME/etc/hadoop

export PATH=$JAVA_HOME/bin:$PATH
export SPARK_HOME=~/spark
export PYTHONPATH=$SPARK_HOME/python:$SPARK_HOME/python/lib/py4j-0.10.9.5-src.zip:$PYTHONPATH
export PATH=$SPARK_HOME/bin:$PATH

export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
export PATH=$JAVA_HOME/bin:$PATH
export SPARK_LOCAL_DIRS=/home/sparkuser/sparkuser/hdfs/yarn/local


# <font color=red> SOURCE bashrc ; REBOOT JUPYTER ; Run section 3</font>

# start dfs, yarn, spark history server

In [20]:
!~/hadoop/bin/hadoop namenode -format


2023-09-23 05:57:22,161 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = devrestricted-binweiyang.ec2.pin220.com/10.9.235.144
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 3.2.4
STARTUP_MSG:   classpath = /home/sparkuser/hadoop/etc/hadoop:/home/sparkuser/hadoop/share/hadoop/common/lib/kerb-client-1.0.1.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/hadoop-annotations-3.2.4.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/commons-configuration2-2.1.1.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/kerb-common-1.0.1.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/kerb-crypto-1.0.1.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/hadoop-auth-3.2.4.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/jersey-core-1.19.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/listenablefuture-9999.0-empty-to-avoid-conflict-with-guava.jar:/home/sparkuser/hadoop/share/hadoop

2023-09-23 05:57:22,236 INFO namenode.NameNode: createNameNode [-format]
2023-09-23 05:57:22,537 INFO common.Util: Assuming 'file' scheme for path /home/sparkuser/sparkuser/hdfs/name in configuration.
2023-09-23 05:57:22,538 INFO common.Util: Assuming 'file' scheme for path /home/sparkuser/sparkuser/hdfs/name in configuration.
Formatting using clusterid: CID-61d40b26-c743-4898-b29a-e5e60d2e2416
2023-09-23 05:57:22,578 INFO namenode.FSEditLog: Edit logging is async:true
2023-09-23 05:57:22,602 INFO namenode.FSNamesystem: KeyProvider: null
2023-09-23 05:57:22,604 INFO namenode.FSNamesystem: fsLock is fair: true
2023-09-23 05:57:22,604 INFO namenode.FSNamesystem: Detailed lock hold time metrics enabled: false
2023-09-23 05:57:22,608 INFO namenode.FSNamesystem: fsOwner             = sparkuser (auth:SIMPLE)
2023-09-23 05:57:22,608 INFO namenode.FSNamesystem: supergroup          = supergroup
2023-09-23 05:57:22,608 INFO namenode.FSNamesystem: isPermissionEnabled = true
2023-09-23 05:57:22,60

In [21]:
!~/hadoop/bin/hadoop datanode -format 


2023-09-23 05:57:30,773 INFO datanode.DataNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting DataNode
STARTUP_MSG:   host = devrestricted-binweiyang.ec2.pin220.com/10.9.235.144
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 3.2.4
STARTUP_MSG:   classpath = /home/sparkuser/hadoop/etc/hadoop:/home/sparkuser/hadoop/share/hadoop/common/lib/kerb-client-1.0.1.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/hadoop-annotations-3.2.4.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/commons-configuration2-2.1.1.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/kerb-common-1.0.1.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/kerb-crypto-1.0.1.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/hadoop-auth-3.2.4.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/jersey-core-1.19.jar:/home/sparkuser/hadoop/share/hadoop/common/lib/listenablefuture-9999.0-empty-to-avoid-conflict-with-guava.jar:/home/sparkuser/hadoop/share/hadoop

Usage: hdfs datanode [-regular | -rollback | -rollingupgrade rollback ]
    -regular                 : Normal DataNode startup (default).
    -rollback                : Rollback a standard or rolling upgrade.
    -rollingupgrade rollback : Rollback a rolling upgrade operation.
  Refer to HDFS documentation for the difference between standard
  and rolling upgrades.

2023-09-23 05:57:30,869 WARN datanode.DataNode: Exiting Datanode
2023-09-23 05:57:30,871 INFO util.ExitUtil: Exiting with status 1: ExitException
2023-09-23 05:57:30,873 INFO datanode.DataNode: SHUTDOWN_MSG: 
/************************************************************
SHUTDOWN_MSG: Shutting down DataNode at devrestricted-binweiyang.ec2.pin220.com/10.9.235.144
************************************************************/


In [ ]:
for l in clients:
    !ssh $l "ps aux | grep java"

In [22]:
!~/hadoop/sbin/start-dfs.sh

Starting namenodes on [devrestricted-binweiyang]
pdsh@devrestricted-binweiyang: devrestricted-binweiyang: rcmd: socket: Permission denied
Starting datanodes
pdsh@devrestricted-binweiyang: localhost: rcmd: socket: Permission denied
Starting secondary namenodes [devrestricted-binweiyang]
pdsh@devrestricted-binweiyang: devrestricted-binweiyang: rcmd: socket: Permission denied


In [ ]:
#skip if we run on single node, we use standalone mode

!~/hadoop/sbin/start-yarn.sh

In [ ]:
!~/hadoop/sbin/stop-all.sh >/dev/null 2>&1
!~/hadoop/sbin/start-all.sh >/dev/null 2>&1

In [ ]:
!scp -o StrictHostKeyChecking=no /home/sparkuser/.ssh/known_hosts {masterip}:~/.ssh/

In [ ]:
!hadoop dfsadmin -safemode leave

In [ ]:
!hadoop fs -mkdir -p /tmp/sparkEventLog

In [ ]:
!cd ~/spark && sbin/start-history-server.sh

In [ ]:
!jps

# install monitor

In [ ]:
for c in hclients:
    !ssh root@$c "apt install sysstat"

## skip emon

In [ ]:
!aws s3 cp s3://mlp-spark-dataset-bucket/sep_private_5_23_linux_12142003820f194.tar.bz2 /home/{user}/

In [ ]:
for c in hclients:
    !scp /home/{user}/sep_private_5_40_linux_042722006ceeae3.tar.bz2 $c:~/

In [ ]:
for c in hclients:
    !ssh {c} bunzip2 sep_private_5_40_linux_042722006ceeae3.tar.bz2

In [ ]:
for c in hclients:
    !ssh {c} tar xvf sep_private_5_40_linux_042722006ceeae3.tar

In [ ]:
for c in hclients:
    !ssh {c} ln -s sep_private_5_40_linux_042722006ceeae3 sep_installed

In [ ]:
for c in hclients:
    !ssh {c} "cd sep_installed; bunzip2 sep_private_5.40_linux_042722006ceeae3.tar.bz2"

In [ ]:
for c in hclients:
    !ssh {c} "cd sep_installed; tar xvf sep_private_5.40_linux_042722006ceeae3.tar"

In [ ]:
for c in hclients:
    !ssh {c} "cd sep_installed/sepdk/src/; echo -e \"\\n\\n\\n\" | ./build-driver"

In [ ]:
for c in hclients:
    !ssh root@{c} "/home/{user}/sep_installed/sepdk/src/insmod-sep -g {user}; "

In [ ]:
for c in hclients:
    !ssh {c} "source /home/{user}/sep_installed/sep_vars.sh > /dev/null 2>&1; emon -v | head -n 1"

In [ ]:
!echo "source /home/sparkuser/sep_installed/sep_vars.sh > /dev/null 2>&1" >> ~/.bashrc

# Inspect CPU Freq & HT

In [ ]:
if is_arm:
    t = r'''
    #include <stdlib.h>
    #include <stdio.h>
    #include <assert.h>
    #include <sys/time.h>
    #include <sched.h>
    #include <sys/shm.h>
    #include <errno.h>
    #include <sys/mman.h>
    #include <unistd.h>                        //used for parsing the command line arguments
    #include <fcntl.h>                        //used for opening the memory device file
    #include <math.h>                        //used for rounding functions
    #include <signal.h>
    #include <linux/types.h>
    #include <stdint.h>

    static inline uint64_t GetTickCount()
    {//Return ns counts
        struct timeval tp;
        gettimeofday(&tp,NULL);
        return tp.tv_sec*1000+tp.tv_usec/1000;
    }

    uint64_t CNT=CNT_DEF;

    int main()
    {

      uint64_t start, end;
      start=end=GetTickCount();

        asm  volatile (
          "1:\n"
          "SUBS %0,%0,#1\n"
          "bne 1b\n"
          ::"r"(CNT)
          );

      end=GetTickCount();

      printf(" total time = %lu, freq = %lu \n", end-start, CNT/(end-start)/1000);

      return 0;
    }
    '''
else:
    t=r'''
    #include <stdlib.h>
    #include <stdio.h>
    #include <assert.h>
    #include <sys/time.h>
    #include <sched.h>
    #include <sys/shm.h>
    #include <errno.h>
    #include <sys/mman.h>
    #include <unistd.h>                        //used for parsing the command line arguments
    #include <fcntl.h>                        //used for opening the memory device file
    #include <math.h>                        //used for rounding functions
    #include <signal.h>
    #include <linux/types.h>
    #include <stdint.h>

    static inline uint64_t GetTickCount()
    {//Return ns counts
        struct timeval tp;
        gettimeofday(&tp,NULL);
        return tp.tv_sec*1000+tp.tv_usec/1000;
    }

    uint64_t CNT=CNT_DEF;

    int main()
    {

      uint64_t start, end;
      start=end=GetTickCount();

        asm  volatile (
          "1:\n"
          "dec %0\n"
          "jnz 1b\n"
          ::"r"(CNT)
          );

      end=GetTickCount();

      printf(" total time = %lu, freq = %lu \n", end-start, CNT/(end-start)/1000);

      return 0;
    }
    '''

In [ ]:
%cd ~
with open("t.c", 'w') as f:
    f.writelines(t)
!gcc -O3 -DCNT_DEF=10000000000LL -o t t.c; gcc -O3 -DCNT_DEF=1000000000000LL -o t.delay t.c;
!for j in `seq 1 $(nproc)`; do echo -n $j; (for i in `seq 1 $j`; do taskset -c $i ./t.delay & done); sleep 1; ./t; killall t.delay; sleep 2; done

# clear yarn cache

In [ ]:
clearcachecmd=";".join([f"rm -rf {l}/{user}/hdfs/yarn/local/usercache/*" for l in datadir])
for c in hclients:
    !ssh {c} {clearcachecmd}

# install gluten

In [ ]:
%cd ~
!rm -rf ~/gluten
!git clone https://github.com/oap-project/gluten.git

In [ ]:
%cd ~/gluten

In [ ]:
!ssh localhost "cd ~/gluten/dev && nohup ./buildbundle-veloxbe.sh --enable_hdfs=ON >build.log 2>&1 &"

In [ ]:
!ls -l ~/gluten/package/target/ | grep bundle

In [ ]:
%cd ep/build-velox/src
!./get_velox.sh --enable_hdfs=ON
!./build_velox.sh --enable_hdfs=ON

In [ ]:
!which java

In [ ]:
# %cd ~/gluten/ep/build-arrow/src/
# ./get_arrow.sh
# ./build_arrow.sh

In [ ]:
%cd ~/gluten/cpp
!mkdir build
%cd build
!cmake -DBUILD_VELOX_BACKEND=ON -DENABLE_HDFS=ON ..
!make -j


In [ ]:
%cd ~/gluten
!mvn clean package -Pbackends-velox -Pspark-3.3 -Pfull-scala-compiler -DskipTests -Dcheckstyle.skip

# Generating data

<font color=red> 
Copy databricks package from another machine:

on sr525 execute

!scp -r -P 22 -o "ProxyCommand nc --proxy-type socks5 --proxy child-prc.intel.com:1080 %h %p" /home/yuzhou/PAUS/gcp_cloud/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar sparkuser@{aws_gluten_ip}:~/
</font>

In [ ]:
import os
!{os.environ['SPARK_HOME']}/sbin/stop-slave.sh
!{os.environ['SPARK_HOME']}/sbin/stop-master.sh


In [ ]:
!{os.environ['SPARK_HOME']}/sbin/start-master.sh

In [ ]:
!{os.environ['SPARK_HOME']}/sbin/start-worker.sh spark://{hostname}:7077 -c {cpu_num}

In [ ]:
!{os.environ['HADOOP_HOME']}/sbin/start-dfs.sh

In [ ]:
!{os.environ['SPARK_HOME']}/sbin/start-history-server.sh

## TPCH

In [ ]:
%cd ~
!git clone https://github.com/databricks/tpch-dbgen.git

In [ ]:
!cd ~/tpch-dbgen && git checkout 0469309147b42abac8857fa61b4cf69a6d3128a8 && make clean && make OS=LINUX

In [ ]:
%cd ~/gluten/backends-velox/workload/tpch/gen_data/parquet_dataset

In [ ]:
#verify parameters
print(scaleFactor)
print(numPartitions)
print(dataformat)

In [ ]:
scala=f'''import com.databricks.spark.sql.perf.tpch._


val scaleFactor = "{scaleFactor}" // scaleFactor defines the size of the dataset to generate (in GB).
val numPartitions = {numPartitions}  // how many dsdgen partitions to run - number of input tasks.

val format = "{dataformat}" // valid spark format like parquet "parquet".
val rootDir = "/tpch_sf1000_zstd" // root directory of location to create data in.
val dbgenDir = "/home/{user}/tpch-dbgen" // location of dbgen

val tables = new TPCHTables(spark.sqlContext,
    dbgenDir = dbgenDir,
    scaleFactor = scaleFactor,
    useDoubleForDecimal = false, // true to replace DecimalType with DoubleType
    useStringForDate = false) // true to replace DateType with StringType


tables.genData(
    location = rootDir,
    format = format,
    overwrite = true, // overwrite the data that is already there
    partitionTables = false, // do not create the partitioned fact tables
    clusterByPartitionColumns = false, // shuffle to get partitions coalesced into single files.
    filterOutNullPartitionValues = false, // true to filter out the partition with NULL key value
    tableFilter = "", // "" means generate all tables
    numPartitions = numPartitions) // how many dsdgen partitions to run - number of input tasks.
'''

with open("tpch_datagen_parquet.scala","w") as f:
    f.writelines(scala)

In [ ]:
#verify parameters
print(num_executors)
print(executor_memory)
print(executor_cores)
print(numPartitions)
print(dataSourceCodec)

In [ ]:
tpch_datagen_parquet=f'''
cat tpch_datagen_parquet.scala | {os.environ['SPARK_HOME']}/bin/spark-shell \
  --num-executors {int(num_executors)} \
  --name tpch_gen_parquet \
  --executor-memory {int(executor_memory)}k \
  --executor-cores {int(executor_cores)} \
  --master spark://{hostname}:7077 \
  --driver-memory 10g \
  --conf spark.executor.memoryOverhead=1g \
  --conf spark.sql.broadcastTimeout=4800 \
  --conf spark.driver.maxResultSize=4g \
  --conf spark.sql.shuffle.partitions={numPartitions} \
  --conf spark.sql.parquet.compression.codec={dataSourceCodec} \
  --conf spark.network.timeout 800s
  --conf spark.executor.heartbeatInterval 200s
  --jars /home/{user}/ipython/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar
'''

with open("tpch_datagen_parquet.sh","w") as f:
    f.writelines(tpch_datagen_parquet)


In [ ]:
!bash tpch_datagen_parquet.sh

In [ ]:
!hadoop fs -du -h /

In [ ]:
!hadoop fs -du -h /tpch_sf1000_zstd

## TPCDS

In [ ]:
%cd ~
!git clone https://github.com/databricks/tpcds-kit.git

In [ ]:
!cd ~/tpcds-kit/tools && make clean && make OS=LINUX CC=gcc-9

In [ ]:
%cd ~/gluten/backends-velox/workload/tpcds/gen_data/parquet_dataset

In [ ]:
scala=f'''import com.databricks.spark.sql.perf.tpcds._


val scaleFactor = "{scaleFactor}" // scaleFactor defines the size of the dataset to generate (in GB).
val numPartitions = {numPartitions}  // how many dsdgen partitions to run - number of input tasks.

val format = "{dataformat}" // valid spark format like parquet "parquet".
val rootDir = "/tpcds_sf1000_zstd" // root directory of location to create data in.
val dsdgenDir = "/home/{user}/tpcds-kit/tools/" // location of dbgen

val tables = new TPCDSTables(spark.sqlContext,
    dsdgenDir = dsdgenDir,
    scaleFactor = scaleFactor,
    useDoubleForDecimal = false, // true to replace DecimalType with DoubleType
    useStringForDate = false) // true to replace DateType with StringType


tables.genData(
    location = rootDir,
    format = format,
    overwrite = true, // overwrite the data that is already there
    partitionTables = true, // create the partitioned fact tables
    clusterByPartitionColumns = true, // shuffle to get partitions coalesced into single files.
    filterOutNullPartitionValues = false, // true to filter out the partition with NULL key value
    tableFilter = "", // "" means generate all tables
    numPartitions = numPartitions) // how many dsdgen partitions to run - number of input tasks.
'''

with open("tpcds_datagen_parquet.scala","w") as f:
    f.writelines(scala)

In [ ]:
tpcds_datagen_parquet=f'''
cat tpcds_datagen_parquet.scala | {os.environ['SPARK_HOME']}/bin/spark-shell \
  --num-executors {int(num_executors)} \
  --name tpch_gen_parquet \
  --executor-memory {int(executor_memory)}k \
  --executor-cores {int(executor_cores)} \
  --master spark://{hostname}:7077 \
  --driver-memory 10g \
  --conf spark.executor.memoryOverhead=1g \
  --conf spark.sql.broadcastTimeout=4800 \
  --conf spark.driver.maxResultSize=4g \
  --conf spark.sql.shuffle.partitions={numPartitions} \
  --conf spark.sql.parquet.compression.codec={dataSourceCodec} \
  --conf spark.network.timeout 800s
  --conf spark.executor.heartbeatInterval 200s
  --jars /home/{user}/ipython/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar
'''

with open("tpcds_datagen_parquet.sh","w") as f:
    f.writelines(tpcds_datagen_parquet)

In [ ]:
!bash tpcds_datagen_parquet.sh

# Run TPC Workload

In [ ]:
%whos